In [118]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import sqlite3
import time
import pandas as pd
all_products=[]
# Function to create the database and table
def create_database():
    conn = sqlite3.connect('flipkart_products.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS products
                 (id INTEGER PRIMARY KEY AUTOINCREMENT,
                  title TEXT,
                  price TEXT,
                  rating TEXT,
                  description TEXT,
                  img_url TEXT)''')
    conn.commit()
    conn.close()

# Function to scrape multiple pages and store data in the database
def scrape_flipkart(start_url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    try:
        driver.get(start_url)
        for i in range(10):
            print("Scraping current page...")
            items = driver.find_elements(By.CLASS_NAME, 'CGtC98')  # Update this if necessary
            for item in items:
                try:
                    title = item.find_element(By.CLASS_NAME, 'KzDlHZ').text  # Update class name
                    price = item.find_element(By.CLASS_NAME, 'hl05eU').text  # Update class name
                    rating = item.find_element(By.CLASS_NAME, 'XQDdHH').text  # Update class name
                    desc =  item.find_element(By.CLASS_NAME, '_6NESgJ').text 
                    all_products.append({
                        'title': title,
                        'price': price,
                        'rating': rating,
                        'description': desc,
                        
                    })
                except Exception:
                    continue
            try:
                next_button = driver.find_element(By.CLASS_NAME, '_1G0WLw')  # Update class name if needed
                next_button.click()
                time.sleep(2)  # Wait for the next page to load
            except NoSuchElementException:
                print("No more pages to scrape.")
                break
    finally:
        driver.quit()
        print("Scraping complete!")

In [198]:
scrape_flipkart('https://www.flipkart.com/search?q=laptop')
print("Scraping completed!")
print(all_products)


Scraping current page...
Scraping current page...
Scraping current page...
Scraping current page...
Scraping current page...
Scraping current page...
Scraping current page...
Scraping current page...
Scraping current page...
Scraping current page...
Scraping complete!
Scraping completed!
[{'title': 'Colorful XS Series Intel Core i5 12th Gen 12500H - (16 GB/512 GB SSD/Windows 11 Home/4 GB Graphics/NVI...', 'price': '₹47,990\n₹89,99946% off', 'rating': '4.6', 'description': 'Intel Core i5 Processor (12th Gen)\n16 GB DDR4 RAM\nWindows 11 Operating System\n512 GB SSD\n39.62 cm (15.6 Inch) Display\n2 Year Carry-in Warranty'}, {'title': 'MSI Cyborg 15 AI PC Intel Core Ultra 7 155H - (16 GB/1 TB SSD/Windows 11 Home/8 GB Graphics/NVIDIA GeF...', 'price': '₹99,990\n₹1,29,99023% off', 'rating': '3.9', 'description': 'Intel Core Ultra 7 Processor\n16 GB DDR5 RAM\nWindows 11 Operating System\n1 TB SSD\n39.62 cm (15.6 Inch) Display\n2 Year Carry-in Warranty'}, {'title': 'Apple MacBook AIR Apple M2 

In [214]:
df=pd.DataFrame(all_products)

In [216]:
df.head()

,title,price,rating,description
0,Colorful XS Series Intel Core i5 12th Gen 1250...,"₹47,990\n₹89,99946% off",4.6,Intel Core i5 Processor (12th Gen)\n16 GB DDR4...
1,MSI Cyborg 15 AI PC Intel Core Ultra 7 155H - ...,"₹99,990\n₹1,29,99023% off",3.9,Intel Core Ultra 7 Processor\n16 GB DDR5 RAM\n...
2,Apple MacBook AIR Apple M2 - (8 GB/256 GB SSD/...,"₹73,990\n₹99,90025% off",4.7,Apple M2 Processor\n8 GB Unified Memory RAM\nM...
3,Apple MacBook AIR Apple M2 - (8 GB/256 GB SSD/...,"₹73,990\n₹99,90025% off",4.7,Apple M2 Processor\n8 GB Unified Memory RAM\nM...
4,DELL Inspiron 3520 Intel Core i3 12th Gen 1215...,"₹33,390\n₹50,52033% off",4.2,Intel Core i3 Processor (12th Gen)\n8 GB DDR4 ...


In [218]:
df = df.assign(
    Discounted=df['price'].str.split('\n').str[0],
    Actual=df['price'].str.split('\n').str[1],
)
df['Discounted']=df['Discounted'].str.replace('₹','')
df['Discount Price']=df['Discounted'].str.replace(',','')
df['Discount Price']=df['Discount Price'].astype(int)
df['Actual']=df['Actual'].str.replace('₹','')
df['Actual']=df['Actual'].str.replace('%','')
df['Actual']=df['Actual'].str.replace(',','')
df['Actual Price']=df['Actual'].str.replace('off','')
df['Actual Price']=df['Actual Price'].astype(int)
df['rating']=df['rating'].astype(float)

In [220]:
df = df.assign(
    Processor=df['description'].str.split('\n').str[0],
    Ram=df['description'].str.split('\n').str[1],
    OS=df['description'].str.split('\n').str[2],
    Storage=df['description'].str.split('\n').str[3],
    
)


In [222]:
df.drop(['description','price','Actual','Discounted'],axis=1,inplace=True)


In [224]:
df.head()

,title,rating,Discount Price,Actual Price,Processor,Ram,OS,Storage
0,Colorful XS Series Intel Core i5 12th Gen 1250...,4.6,47990,8999946,Intel Core i5 Processor (12th Gen),16 GB DDR4 RAM,Windows 11 Operating System,512 GB SSD
1,MSI Cyborg 15 AI PC Intel Core Ultra 7 155H - ...,3.9,99990,12999023,Intel Core Ultra 7 Processor,16 GB DDR5 RAM,Windows 11 Operating System,1 TB SSD
2,Apple MacBook AIR Apple M2 - (8 GB/256 GB SSD/...,4.7,73990,9990025,Apple M2 Processor,8 GB Unified Memory RAM,Mac OS Operating System,256 GB SSD
3,Apple MacBook AIR Apple M2 - (8 GB/256 GB SSD/...,4.7,73990,9990025,Apple M2 Processor,8 GB Unified Memory RAM,Mac OS Operating System,256 GB SSD
4,DELL Inspiron 3520 Intel Core i3 12th Gen 1215...,4.2,33390,5052033,Intel Core i3 Processor (12th Gen),8 GB DDR4 RAM,64 bit Windows 11 Operating System,512 GB SSD
